https://web.archive.org/web/20160417083958/https://coinmarketcap.com/exchanges/volume/24-hour/

In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
from dateutil.relativedelta import relativedelta
import datetime
import pandas as pd

In [34]:
def getStats(datestr):
    result = requests.get("https://web.archive.org/web/%s/https://coinmarketcap.com/exchanges/volume/24-hour/" % datestr)
    soup = BeautifulSoup(result.content)
    samples = soup.find_all("td", "bold text-right volume")
    usds = []
    for sample in samples:
        usds.append(int(sample.text.replace("$","").replace(" ", "").replace(",", "").replace("?", "0").strip()))
    usds = np.array(usds)
#     print usds
    ## logging the stats
    num_exchange = float(len(usds))
    total = float(np.sum(usds))
    top_10_total = float(np.sum(usds[usds.argsort()[-10:]][::-1]))
    top_10_percent_total = float(np.sum(usds[usds.argsort()[-int(num_exchange/10):]][::-1])) 
    num_75_market_share = np.sum((np.cumsum(usds) >= np.sum(usds) * 0.75) * -1 + 1) + 1
    num_95_market_share = np.sum((np.cumsum(usds) >= np.sum(usds) * 0.95) * -1 + 1) + 1
    num_99_market_share = np.sum((np.cumsum(usds) >= np.sum(usds) * 0.99) * -1 + 1) + 1
    return [top_10_percent_total, top_10_total, total, num_exchange, num_75_market_share, num_95_market_share, num_99_market_share]

In [35]:
today = datetime.date.today()
current = datetime.date(2015, 1, 1)  

dates = []
top_10_percent_totals = []
top_10_totals = []
totals = []
num_exchanges = []
num_75_market_shares = []
num_95_market_shares = []
num_99_market_shares = []

while current <= today:
    datestr = current.strftime('%Y%m%d000000')
    date = current.strftime('%Y-%m-%d')
    top_10_percent_total, top_10_total, total, num_exchange, num_75_market_share, num_95_market_share, num_99_market_share = getStats(datestr)

    current += relativedelta(months=1)
    dates.append(date)
    top_10_percent_totals.append(top_10_percent_total)
    top_10_totals.append(top_10_total)
    totals.append(total)
    num_exchanges.append(num_exchange)
    num_75_market_shares.append(num_75_market_share)
    num_95_market_shares.append(num_95_market_share)
    num_99_market_shares.append(num_99_market_share)

In [36]:
df = pd.DataFrame({'date': dates, 'top_10_percent_total': top_10_percent_totals, 'top_10_total': top_10_totals, 
                   'total': totals, 'num_exchange': num_exchanges, 'num_75_market_share': num_75_market_shares, 
                  'num_95_market_share': num_95_market_shares, 'num_99_market_share': num_99_market_shares})
df['market_share_top_10'] = df['top_10_total'] / df['total']
df['market_share_top_10_percent'] = df['top_10_percent_total'] / df['total']
df['avg_volume_per_exchange'] = df['total'] / df['num_exchange']

In [37]:
df

,date,num_75_market_share,num_95_market_share,num_99_market_share,num_exchange,top_10_percent_total,top_10_total,total,market_share_top_10,market_share_top_10_percent,avg_volume_per_exchange
0,2015-01-01,3,11,18,53.0,1.552188e+07,1.732464e+07,1.834898e+07,0.944175,0.845926,3.462071e+05
1,2015-02-01,3,8,17,56.0,2.267768e+07,2.416061e+07,2.502700e+07,0.965382,0.906129,4.469107e+05
2,2015-03-01,3,7,15,55.0,5.046482e+07,5.300492e+07,5.452981e+07,0.972036,0.925454,9.914511e+05
3,2015-04-01,3,11,17,51.0,1.280112e+07,1.388974e+07,1.466778e+07,0.946956,0.872738,2.876035e+05
4,2015-05-01,4,14,23,53.0,1.104753e+07,1.248511e+07,1.358415e+07,0.919094,0.813266,2.563046e+05
5,2015-06-01,5,13,23,58.0,1.473583e+07,1.653170e+07,1.773163e+07,0.932328,0.831048,3.057178e+05
6,2015-07-01,6,15,24,57.0,1.459978e+07,1.809790e+07,1.988987e+07,0.909905,0.734031,3.489451e+05
7,2015-08-01,3,13,23,62.0,3.669130e+07,3.845744e+07,4.112246e+07,0.935193,0.892245,6.632655e+05
8,2015-09-01,6,15,24,62.0,1.668089e+07,1.836331e+07,2.025606e+07,0.906559,0.823502,3.267106e+05
9,2015-10-01,7,19,28,60.0,6.218545e+06,7.413913e+06,8.741823e+06,0.848097,0.711356,1.456970e+05


In [39]:
df.to_csv("exchange_fragmentation.csv")